In [5]:
import torch
from src.evaluate import evaluate_single
from src.translate import translate_sentence

from src.dataset import get_tokenizers
from src.vocab import build_vocab
from src.utils import set_seed
from src.models.encoder import Encoder
from src.models.decoder import Decoder
from src.models.seq2seq import Seq2Seq

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(42)


ModuleNotFoundError: No module named 'torch'

In [6]:
tokenizer_en, tokenizer_fr = get_tokenizers()

# Load vocab đã build lúc train (hoặc build lại từ train dataset)
from src.dataset import load_parallel

import pickle

with open("checkpoints/vocab.pkl", "rb") as f:
    vocab = pickle.load(f)

src_stoi = vocab["src_stoi"]
src_itos = vocab["src_itos"]
trg_stoi = vocab["trg_stoi"]
trg_itos = vocab["trg_itos"]



NameError: name 'get_tokenizers' is not defined

In [ ]:
encoder = Encoder(
    vocab_size=len(src_itos),
    embed_dim=256,
    hidden_size=512,
    num_layers=3,
    dropout=0.3,
    pad_idx=src_stoi["<pad>"]
)

decoder = Decoder(
    vocab_size=len(trg_itos),
    embed_dim=256,
    hidden_size=512,
    num_layers=3,
    dropout=0.3,
    pad_idx=trg_stoi["<pad>"]
)

model = Seq2Seq(
    encoder,
    decoder,
    device,
    sos_idx=trg_stoi["<sos>"],
    eos_idx=trg_stoi["<eos>"],
    pad_idx=trg_stoi["<pad>"],
    max_len=50
).to(device)

checkpoint = torch.load("checkpoints/best_model.pth", map_location=device)
model.load_state_dict(checkpoint["model_state"])


In [ ]:
sentence = "this is a very simple test sentence."

translated = translate_sentence(
    sentence,
    model,
    tokenizer_en,
    src_stoi,
    trg_itos,
    device,
    beam_size=5   # hoặc None
)

print("SRC:", sentence)
print("TRG:", translated)
